In [1]:

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import auth
import json
import requests
import Auth
from firebase_admin import firestore

In [2]:
cred = credentials.Certificate("F:\\personal project'\\backend\\site-hopper-adminsdk.json")

firebase_admin.initialize_app(cred)


In [3]:
db = firestore.client()

In [4]:
users_ref = db.collection(u'users')
docs = users_ref.stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')
  

uid1 => {'name': 'Tushar'}
uid2 => {'name': 'Akshay'}
uid3 => {'name': 'ellon'}


In [5]:
# Note: Use of CollectionRef stream() is prefered to get()
docs = db.collection(u'domains').stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

stackoverflow.com => {}
youtube.com => {}


### Get top comments for specific post in specific domain order by rating 


In [6]:
query = db.collection('domains').document('stackoverflow.com').collection("routes").document("question::1").collection("comments").order_by("rating").limit(3)
docs = query.stream()
for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

uid2 => {'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 40, tzinfo=<UTC>), 'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E627A2B608>, 'rating': 1, 'body': 'I also dont know', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E627A2BE88>}
uid1 => {'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 30, tzinfo=<UTC>), 'list_reply': [<google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E608>, <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E908>], 'rating': 3, 'body': 'what is this', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E248>}
uid3 => {'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 41, tzinfo=<UTC>), 'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985CFC8>, 'rating': 5, 'body': 'lets go in space', 'user': <google.cloud.firestore_v1.document.DocumentReference objec

### Get top reply for specific comment order by time

In [7]:
doc = db.collection('domains').document('stackoverflow.com')\
.collection("routes").document("question::1").collection("comments").document("uid1")

comment_dict = doc.get().to_dict()
list_reply = comment_dict["list_reply"]
for reply in list_reply[:1]:
    print(reply.get().to_dict()["body"])


I also dont know


### Get top comments and reply by specific user for all posts in all domain.

In [32]:
doc = db.collection('users').document('uid2')

user_dict = doc.get().to_dict()
list_comment = user_dict["list_comment"]

for comment in list_comment[:3]:
    parent = comment.get().to_dict().get("parent",None)
    if parent is not None:
        print("parent : "+parent.get().to_dict()["body"])
        
    
    print("Body : "+comment.get().to_dict()["body"])
    print(comment.get().to_dict())

parent : what is this
Body : I also dont know
{'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 40, tzinfo=<UTC>), 'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E348>, 'rating': 1, 'body': 'I also dont know', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E188>}
parent : something is fishy
Body : you are dumb
{'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5A88>, 'rating': 5, 'body': 'you are dumb', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5D08>}
parent : this is awesome
Body : yes it is
{'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5C08>, 'rating': 4, 'body': 'yes it is', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5B48>}


3